In [1]:
from pprint import pprint
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Experimental information

*  xkappa biotin spots: these spots are coated with human kappa light chain conjugated to biotin, so that when the streptavidin-HRP is added there is strong signal independent of the serum content.  **We should not expect the fiducial signal to weaken with serial dilution.**
*  anti-His tag spots: All of the recombinant proteins that are used to coat the plates have hexa-His tags cloned into them, and the coat protein in these spots is anti-His. In this experiment, all of the recombinant antigens were spotted here on the anti-His coated spot, then serum was added so that the spot is representative of a polyclonal response. There was more cross-reactivity than expected and **this control will not be used in the future covid plates.**

# Data format conventions

For ease of reading and manipulating data, all data is converted in $n$-dimensional arrays. 
* Well metadata is #wells $\times$ 4 array. 
* antigen metadata is  #rows $\times$ #columns in the array.
* background corrected ODs is a 4D float array (Well row x Well column x Array row x Array column).

# Read identity of wells and antigen spots in 2D arrays

In [35]:
plateinfo_path=r'/Volumes/GoogleDrive/My Drive/ELISAarrayReader/images_scienion/Plates_given_to_manu/2020-01-15_plate4_AEP_Feb3_6mousesera/PlateInfo.csv'
antigenOD_path=r'/Volumes/GoogleDrive/My Drive/ELISAarrayReader/images_scienion/Plates_given_to_manu/2020-01-15_plate4_AEP_Feb3_6mousesera/20200327_analyzedPlate/ODs_2020_327_258.xlsx'

In [36]:
plateinfo_dataframe=pd.read_csv(plateinfo_path)
pprint(plateinfo_dataframe)
platinfo=plateinfo_dataframe.to_numpy()[:,1:]
pprint(platinfo)

   Well     Sera ID              Type  Dilution
0    A1     Mouse 1        Diagnostic      1:50
1    A2     Mouse 1        Diagnostic      1:50
2    A3     Mouse 2        Diagnostic      1:50
3    A4     Mouse 2        Diagnostic      1:50
4    A5     Mouse 3        Diagnostic      1:50
..  ...         ...               ...       ...
91   H8  MAb-biotin  control/standard   0.41 nM
92   H9  MAb-biotin  control/standard  0.136 nM
93  H10  MAb-biotin  control/standard  0.045 nM
94  H11  MAb-biotin  control/standard  0.015 nM
95  H12         NaN               NaN       NaN

[96 rows x 4 columns]
array([['Mouse 1', 'Diagnostic', '1:50'],
       ['Mouse 1', 'Diagnostic', '1:50'],
       ['Mouse 2', 'Diagnostic', '1:50'],
       ['Mouse 2', 'Diagnostic', '1:50'],
       ['Mouse 3', 'Diagnostic', '1:50'],
       ['Mouse 3', 'Diagnostic', '1:50'],
       ['Mouse 4', 'Diagnostic', '1:50'],
       ['Mouse 4', 'Diagnostic', '1:50'],
       ['Mouse 5', 'Diagnostic', '1:50'],
       ['Mouse 5', 'Dia

In [37]:
converter = {col: str for col in range(0,7)}
antigens_dataframe=pd.read_excel(antigenOD_path, sheet_name='antigens', converters=converter)
antigens=antigens_dataframe.to_numpy()[:,1:] 
pprint(antigens)

array([['xkappa-biotin', '114', '100', 'KZ52', 'c13C6', 'c2G4', 'c4G7',
        'xkappa-biotin'],
       ['xkappa-biotin', '114', '100', 'KZ52', 'c13C6', 'c2G4', 'c4G7',
        'xmouse-IgG'],
       ['anti-HIS tag', '15731', '15742', '15750', '15878', '15946',
        '15960', 'Q411'],
       ['anti-HIS tag', '15731', '15742', '15750', '15878', '15946',
        '15960', 'Q411'],
       ['anti-HIS tag', '15974', '16061', 'FVM04', 'VIC122', 'Q206',
        'Q314', 'GFP-foldon'],
       ['xkappa-biotin', '15974', '16061', 'FVM04', 'VIC122', 'Q206',
        'Q314', 'xkappa-biotin']], dtype=object)


# Read ODs of all wells in a 4D array (Well row x Well column x Array row x Array column)

## Read data

In [14]:
row_label=['A','B','C','D','E','F','G','H']
col_label=[str(idx) for idx in np.arange(12)+1]
antigenOD=np.empty((8,12,6,8)) # 96 wells, 6 antigen rows, 8 antigen columns
wellID=np.empty((8,12),dtype=object)

In [15]:
# Read all wells into dictionary and into a 4D numpy array.
for r in np.arange(8):
    for c in np.arange(12):
        well_key = row_label[r]+col_label[c]
        antigenOD_dataframe=pd.read_excel(antigenOD_path, sheet_name=well_key)
        antigenOD[r,c,:,:]=antigenOD_dataframe.to_numpy()[:,1:] # First column is simply the index. to_numpy returns 6x9 array.
        wellID[r,c] = well_key

## how to index the OD data.

#### direct indexing

In [19]:
pprint(antigenOD[0,0]) # Well A1

array([[1.48690358,        nan, 1.88054858, 2.58605292, 1.28097541,
        2.65749052, 1.4895249 , 1.55835905],
       [1.46043415, 1.85433613, 2.27518306, 2.74934729, 1.43469453,
        2.05008816, 1.69365441, 3.02854479],
       [3.58943717, 1.60508459, 2.58859898, 1.26907949,        nan,
        1.27786073, 1.82586891, 1.66100972],
       [3.3421515 , 1.81320625, 2.73909223, 1.27429019, 2.60367051,
               nan, 2.31202412, 1.60264153],
       [3.43131823, 2.24281023, 3.27112586, 2.11036309, 2.43932369,
        2.12406351,        nan, 4.61593211],
       [1.46372823, 2.1719292 , 3.20397848, 2.05375365, 2.36963313,
        2.01548414, 1.32053514, 1.4020999 ]])


In [18]:
pprint(antigenOD[0,3]) # well A4

array([[       nan, 1.3451414 , 2.99774562,        nan, 1.29873839,
               nan, 1.23414691, 1.61283121],
       [1.44550936,        nan, 2.96651674, 1.26165007,        nan,
               nan,        nan, 3.55750328],
       [1.87884234, 1.28353302,        nan,        nan,        nan,
               nan,        nan,        nan],
       [1.93024336,        nan, 1.16157532,        nan,        nan,
               nan,        nan,        nan],
       [1.74705778,        nan,        nan,        nan, 1.1752757 ,
               nan,        nan,        nan],
       [1.44784659,        nan,        nan, 1.23790682,        nan,
               nan,        nan, 1.40577401]])


#### logical indexing

In [32]:
pprint(antigenOD[wellID == 'A4'])

array([[[       nan, 1.3451414 , 2.99774562,        nan, 1.29873839,
                nan, 1.23414691, 1.61283121],
        [1.44550936,        nan, 2.96651674, 1.26165007,        nan,
                nan,        nan, 3.55750328],
        [1.87884234, 1.28353302,        nan,        nan,        nan,
                nan,        nan,        nan],
        [1.93024336,        nan, 1.16157532,        nan,        nan,
                nan,        nan,        nan],
        [1.74705778,        nan,        nan,        nan, 1.1752757 ,
                nan,        nan,        nan],
        [1.44784659,        nan,        nan, 1.23790682,        nan,
                nan,        nan, 1.40577401]]])


In [33]:
pprint(antigenOD[wellID == 'A1'])

array([[[1.48690358,        nan, 1.88054858, 2.58605292, 1.28097541,
         2.65749052, 1.4895249 , 1.55835905],
        [1.46043415, 1.85433613, 2.27518306, 2.74934729, 1.43469453,
         2.05008816, 1.69365441, 3.02854479],
        [3.58943717, 1.60508459, 2.58859898, 1.26907949,        nan,
         1.27786073, 1.82586891, 1.66100972],
        [3.3421515 , 1.81320625, 2.73909223, 1.27429019, 2.60367051,
                nan, 2.31202412, 1.60264153],
        [3.43131823, 2.24281023, 3.27112586, 2.11036309, 2.43932369,
         2.12406351,        nan, 4.61593211],
        [1.46372823, 2.1719292 , 3.20397848, 2.05375365, 2.36963313,
         2.01548414, 1.32053514, 1.4020999 ]]])


# Control wells.